## Factoring Integers

The most famous application of quantum computers is factoring integers using Shor's algorithm. It is of considerate interest because an efficient algorithm for factoring would allow to crack modern asymmetric encryption schemes, such as RSA.

For small integers this quantum algorithm can be simulated on a classical computer.

In [ ]:
#!pip install qiskit[all]

Here we need a set of additional tools and helper functions, the actual algorithm will follow further down.

In [1]:
from math import gcd, log2, ceil
import numpy as np
from random import randint
import sys
sys.path.append('./GTC_tutorial/data')
from qiskit.algorithms import Shor
import qiskit_to_cirq_conversion
import fractions


In [2]:
def get_num_bits(N):
    return int(log2(N)+1)


In [3]:
def get_histogram(result, width, shots):
    if shots == 1:
        return {result.tobytes(): 1}

    # We rely on the PY37+ feature that the dict is ordered, and sort the
    # measurement outcome (significant ones come first)
    out = {}
    axis = None if result.ndim == 1 else 0
    keys, values = np.unique(result, axis=axis, return_counts=True)
    idx = np.argsort(values)[::-1]
    keys = keys[idx]
    values = values[idx]
    for i in range(keys.shape[0]):
        # numpy arrays are not hashable
        if result.ndim == 2:  # from qsimcirq
            k = ''.join(str(keys[i])[1:-1].split())
        else:
            assert False
        out[k] = values[i]

    # sanity check
    counts = 0
    for k, v in out.items():
        counts += v
    assert counts == shots, f"counts = {counts}, shots = {shots}"

    if shots/len(out) < 5.0:
        print(f"WARNING: too many ({len(out)}) unique bitstrings with limited shots ({shots}), "
              "statistics may not be accurate")

    return out


In [4]:
def get_factors(r, r_nbits, x, N):
    assert r_nbits > 0
    assert x > 0
    assert N > 0
    eigenphase = float(r) / 2**r_nbits
    f = fractions.Fraction.from_float(eigenphase).limit_denominator(N)    
    # If the numerator is zero, the order finder failed.
    if f.numerator == 0:
        return None
    
    # Else, return the denominator if it is valid.
    r = f.denominator
    if x**r % N != 1:
        return None
    return r

### Shor's algorithm
The integer factoring problem is finding the components $g$, $h$ of a given product $N=g \cdot h$ of two primes.

The algorithm consists of two ideas.

##### 1. Reduce the problem of factoring the integer to the order-finding problem (classically):
Given an integer $a$ find the period $x=r$ of $a^x (\text{mod} N)$, <br>
i.e., the smallest positive integer $r$ that satisfies $a^r = 1 (\text{mod} N)$.

In [5]:
def shors_algorithm(N):
    print("Running Shor's algorithm classically on N =", N)
    while(True):
        # 1. Select a random integer between 2 and N-1
        a = randint(2,N-1)
        
        # 2. See if it happens to factor N
        print("Trying a =", a)
        g = gcd(a,N)
        if g != 1:
            print("Found factor by chance:", g)
            return (g, N//g)
        
        # 3. Find the period a^r = 1 (mod N)
        r = find_period(a, N)
        
        # 4. If a period is found and it is
        # * even and
        # * not a^(r/2) = -1 (mod N),
        # the factors will be the gcd(a^(r/2) + 1, N), gcd(a^(r/2 - 1, N).
        if r != None and r % 2 != 1 and (r//2) % N != N-1:
            a1 = gcd(a ** (r//2) + 1, N)
            print("Found factor:", a1)
            return (a1, N//a1)
        print("retrying...")

##### 2. Solve the order-finding problem with a quantum algorithm.

The key component of the algorithm is an efficient quantum algorithm to find the period $r$ of the operation $x^r (\text{mod} N)$. A naive classical algorithm for this problem is easy to implement, but inefficient:

In [6]:
def find_period(a, N):
    return find_period_classical(a, N)

In [7]:
def find_period_classical(a, N):
    """A naive classical method to find the period r of x^r (mod N)."""
    assert 1 < a and a < N
    r = 1
    y = a
    while y != 1:
        y = y * a % N
        r += 1
    return r

In [8]:
my_integer = 3*5
shors_algorithm(my_integer)

Running Shor's algorithm classically on N = 15
Trying a = 10
Found factor by chance: 5


(5, 3)

An efficient quantum solution derives the period from a measurement of $n = \lceil log2(N) \rceil$ qubits:

In [9]:
def find_period_quantum(x, N):
    """The quantum algorithm to find the period r of x^r (mod N)."""
    
    print("Running quantum algorithm...")
    shots   = 128
    nbits   = get_num_bits(N)
    
    qiskit_circuit              = Shor().construct_circuit(N, x)
    cirq_circuit, qubit_offsets = qiskit_to_cirq_conversion.convert(qiskit_circuit)

    print("Total number of qubits:", qubit_offsets['total'])
    cirq_circuit.append(cirq.measure_each(*cirq.LineQubit.range(qubit_offsets['up'], qubit_offsets['down'])))

    result = qsim_simulator.sample(cirq_circuit,repetitions=shots)
    measurement_results = result.to_numpy()
    print("Measurement results:")
    print(measurement_results)
    hist = get_histogram(measurement_results, qubit_offsets['down'], shots)
    print("Number of occurences:", hist)
    most_probable_bitpattern = max(hist, key=hist.get)
    result = int(most_probable_bitpattern[::-1],2)
    print("Most probable bitpattern:", most_probable_bitpattern, "=", result)
    r = get_factors(result, qubit_offsets['down'], x, N)
    print("Found period:", r)
    print("The circuit:")
    print(cirq_circuit)
    return r

In [10]:
def find_period(a, N):
    return find_period_quantum(a, N)

In [12]:
my_integer = 3*5
shors_algorithm(my_integer)

Running Shor's algorithm classically on N = 15
Trying a = 7
Running quantum algorithm...
Total number of qubits: 18


NameError: name 'cirq' is not defined